In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib    
import matplotlib.pyplot as plt

import skimage.io

import datetime

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

#from samples.balloon import balloon
from bin import crack

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
# COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

datasetName = "Deepcrack"
model_path = "../logs/mask_rcnn_cracks_{}.h5".format(datasetName)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")

Using TensorFlow backend.


Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the CocoConfig class in coco.py.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the CocoConfig class and override the attributes you need to change.

In [2]:
class InferenceConfig(crack.CracksConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0.0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE            

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on Deepcrack
model.load_weights(model_path, by_name=True)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








Class Names
"""
The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our Dataset class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our Dataset class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the class_names property like this.

# Load Custom dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)
"""

In [4]:
# Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'crack']

In [5]:
from matplotlib import patches,  lines


def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:,:,n] = np.where(
            mask==1,
            image[:,:,n]*(1-alpha) + alpha*c,
            image[:,:,n]
        )
    return image


def display_instances(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        print("\n*** boxes ***\n")
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]
    """
    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True
    """
    # Generate random colors
    colors = visualize.random_colors(N)

    # Show area outside image boundaries.
    # height, width = image.shape[:2]
    """
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)"""

    #masked_image = image.astype(np.uint32).copy()
    
    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue    
        
        y1, x1, y2, x2 = boxes[i]
        label = class_names[class_ids[i]]
        score = scores[i] if scores is not None else None
        caption = "{} {:.2f}".format(label, score) if score else label
        mask = masks[:,:,i]
        
        image = apply_mask(image, mask, color)
        image = np.array(image)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
     
    return image

In [6]:
import cv2

print(VIDEO_DIR)
video_list = os.listdir(VIDEO_DIR)
print(video_list)
video_name = video_list[0]
video_path = os.path.join(VIDEO_DIR, video_name) 

video_name = os.path.splitext(video_name)

# Video capture
vcapture = cv2.VideoCapture(video_path)
width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = vcapture.get(cv2.CAP_PROP_FPS)

# Define codec and create video writer
dst_file_name = "results/{}_{}_{:%Y%m%dT%H%M%S}.avi".format(datasetName, video_name[0], datetime.datetime.now())
dst_file_path = os.path.join(VIDEO_DIR, dst_file_name)
vwriter = cv2.VideoWriter(dst_file_path,
                        cv2.VideoWriter_fourcc(*'MJPG'),
                        fps, (width, height))

count = 0
success = True

while success:
    print("frame: ", count)
    # Read next image
    success, image = vcapture.read()
    if success:
        # OpenCV returns images as BGR, convert to RGB
        image = image[..., ::-1]
        # Detect objects
        r = model.detect([image], verbose=0)[0]
        # Color splash
        #splash = color_splash(image, r['masks'])
        
        image = display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
        # RGB -> BGR to save image to video
        #splash = splash[..., ::-1]
        # Add image to video writer
        #vwriter.write(splash)
        vwriter.write(image)
        count += 1
        
vwriter.release()
print("Saved to ", dst_file_path)
    

/data/jw/crack_ws/src/Mask_RCNN/videos
['1st_test_left.mp4', '1st_test_right.mp4', '2nd_test_left.mp4', '2nd_test_right.mp4', '3rd_test_left.avi', '3rd_test_right.avi', '20201005_173852.mp4', '20201005_173859.mp4', '20201005_174007.mp4', '20201005_174049.mp4', 'results']
frame:  0

*** No instances to display *** 

frame:  1

*** No instances to display *** 

frame:  2

*** No instances to display *** 

frame:  3

*** No instances to display *** 

frame:  4

*** No instances to display *** 

frame:  5

*** No instances to display *** 

frame:  6

*** No instances to display *** 

frame:  7

*** No instances to display *** 

frame:  8

*** No instances to display *** 

frame:  9

*** No instances to display *** 

frame:  10

*** No instances to display *** 

frame:  11

*** No instances to display *** 

frame:  12

*** No instances to display *** 

frame:  13

*** No instances to display *** 

frame:  14

*** No instances to display *** 

frame:  15

*** No instances to display *** 

fr


*** No instances to display *** 

frame:  172

*** No instances to display *** 

frame:  173

*** No instances to display *** 

frame:  174

*** No instances to display *** 

frame:  175

*** No instances to display *** 

frame:  176

*** No instances to display *** 

frame:  177

*** No instances to display *** 

frame:  178

*** No instances to display *** 

frame:  179

*** No instances to display *** 

frame:  180

*** No instances to display *** 

frame:  181

*** No instances to display *** 

frame:  182

*** No instances to display *** 

frame:  183

*** No instances to display *** 

frame:  184

*** No instances to display *** 

frame:  185

*** No instances to display *** 

frame:  186

*** No instances to display *** 

frame:  187

*** No instances to display *** 

frame:  188

*** No instances to display *** 

frame:  189

*** No instances to display *** 

frame:  190

*** No instances to display *** 

frame:  191

*** No instances to display *** 

frame:  192

*** No insta


*** No instances to display *** 

frame:  349

*** No instances to display *** 

frame:  350

*** No instances to display *** 

frame:  351

*** No instances to display *** 

frame:  352

*** No instances to display *** 

frame:  353

*** No instances to display *** 

frame:  354

*** No instances to display *** 

frame:  355

*** No instances to display *** 

frame:  356

*** No instances to display *** 

frame:  357

*** No instances to display *** 

frame:  358

*** No instances to display *** 

frame:  359

*** No instances to display *** 

frame:  360

*** No instances to display *** 

frame:  361

*** No instances to display *** 

frame:  362

*** No instances to display *** 

frame:  363

*** No instances to display *** 

frame:  364

*** No instances to display *** 

frame:  365

*** No instances to display *** 

frame:  366

*** No instances to display *** 

frame:  367

*** No instances to display *** 

frame:  368

*** No instances to display *** 

frame:  369

*** No insta


*** No instances to display *** 

frame:  526

*** No instances to display *** 

frame:  527

*** No instances to display *** 

frame:  528

*** No instances to display *** 

frame:  529

*** No instances to display *** 

frame:  530

*** No instances to display *** 

frame:  531

*** No instances to display *** 

frame:  532

*** No instances to display *** 

frame:  533

*** No instances to display *** 

frame:  534

*** No instances to display *** 

frame:  535

*** No instances to display *** 

frame:  536

*** No instances to display *** 

frame:  537

*** No instances to display *** 

frame:  538

*** No instances to display *** 

frame:  539

*** No instances to display *** 

frame:  540

*** No instances to display *** 

frame:  541

*** No instances to display *** 

frame:  542

*** No instances to display *** 

frame:  543

*** No instances to display *** 

frame:  544

*** No instances to display *** 

frame:  545

*** No instances to display *** 

frame:  546

*** No insta


*** No instances to display *** 

frame:  703

*** No instances to display *** 

frame:  704

*** No instances to display *** 

frame:  705

*** No instances to display *** 

frame:  706

*** No instances to display *** 

frame:  707

*** No instances to display *** 

frame:  708

*** No instances to display *** 

frame:  709

*** No instances to display *** 

frame:  710

*** No instances to display *** 

frame:  711

*** No instances to display *** 

frame:  712

*** No instances to display *** 

frame:  713

*** No instances to display *** 

frame:  714

*** No instances to display *** 

frame:  715

*** No instances to display *** 

frame:  716

*** No instances to display *** 

frame:  717

*** No instances to display *** 

frame:  718

*** No instances to display *** 

frame:  719

*** No instances to display *** 

frame:  720

*** No instances to display *** 

frame:  721

*** No instances to display *** 

frame:  722

*** No instances to display *** 

frame:  723

*** No insta

...


*** No instances to display *** 

frame:  10457

*** No instances to display *** 

frame:  10458

*** No instances to display *** 

frame:  10459

*** No instances to display *** 

frame:  10460

*** No instances to display *** 

frame:  10461

*** No instances to display *** 

frame:  10462

*** No instances to display *** 

frame:  10463

*** No instances to display *** 

frame:  10464

*** No instances to display *** 

frame:  10465

*** No instances to display *** 

frame:  10466

*** No instances to display *** 

frame:  10467

*** No instances to display *** 

frame:  10468

*** No instances to display *** 

frame:  10469

*** No instances to display *** 

frame:  10470

*** No instances to display *** 

frame:  10471

*** No instances to display *** 

frame:  10472

*** No instances to display *** 

frame:  10473

*** No instances to display *** 

frame:  10474

*** No instances to display *** 

frame:  10475

*** No instances to display *** 

frame:  10476

*** No instances to


*** No instances to display *** 

frame:  10625

*** No instances to display *** 

frame:  10626

*** No instances to display *** 

frame:  10627

*** No instances to display *** 

frame:  10628

*** No instances to display *** 

frame:  10629

*** No instances to display *** 

frame:  10630

*** No instances to display *** 

frame:  10631

*** No instances to display *** 

frame:  10632

*** No instances to display *** 

frame:  10633

*** No instances to display *** 

frame:  10634

*** No instances to display *** 

frame:  10635

*** No instances to display *** 

frame:  10636

*** No instances to display *** 

frame:  10637

*** No instances to display *** 

frame:  10638

*** No instances to display *** 

frame:  10639

*** No instances to display *** 

frame:  10640

*** No instances to display *** 

frame:  10641

*** No instances to display *** 

frame:  10642

*** No instances to display *** 

frame:  10643

*** No instances to display *** 

frame:  10644

*** No instances to


*** No instances to display *** 

frame:  10793

*** No instances to display *** 

frame:  10794

*** No instances to display *** 

frame:  10795

*** No instances to display *** 

frame:  10796

*** No instances to display *** 

frame:  10797

*** No instances to display *** 

frame:  10798

*** No instances to display *** 

frame:  10799

*** No instances to display *** 

frame:  10800

*** No instances to display *** 

frame:  10801

*** No instances to display *** 

frame:  10802

*** No instances to display *** 

frame:  10803

*** No instances to display *** 

frame:  10804

*** No instances to display *** 

frame:  10805

*** No instances to display *** 

frame:  10806

*** No instances to display *** 

frame:  10807

*** No instances to display *** 

frame:  10808

*** No instances to display *** 

frame:  10809

*** No instances to display *** 

frame:  10810

*** No instances to display *** 

frame:  10811

*** No instances to display *** 

frame:  10812

*** No instances to


*** No instances to display *** 

frame:  10961

*** No instances to display *** 

frame:  10962

*** No instances to display *** 

frame:  10963

*** No instances to display *** 

frame:  10964

*** No instances to display *** 

frame:  10965

*** No instances to display *** 

frame:  10966

*** No instances to display *** 

frame:  10967

*** No instances to display *** 

frame:  10968

*** No instances to display *** 

frame:  10969

*** No instances to display *** 

frame:  10970

*** No instances to display *** 

frame:  10971

*** No instances to display *** 

frame:  10972

*** No instances to display *** 

frame:  10973

*** No instances to display *** 

frame:  10974

*** No instances to display *** 

frame:  10975

*** No instances to display *** 

frame:  10976

*** No instances to display *** 

frame:  10977

*** No instances to display *** 

frame:  10978

*** No instances to display *** 

frame:  10979

*** No instances to display *** 

frame:  10980

*** No instances to


*** No instances to display *** 

frame:  11129

*** No instances to display *** 

frame:  11130

*** No instances to display *** 

frame:  11131

*** No instances to display *** 

frame:  11132

*** No instances to display *** 

frame:  11133

*** No instances to display *** 

frame:  11134

*** No instances to display *** 

frame:  11135

*** No instances to display *** 

frame:  11136

*** No instances to display *** 

frame:  11137

*** No instances to display *** 

frame:  11138

*** No instances to display *** 

frame:  11139

*** No instances to display *** 

frame:  11140

*** No instances to display *** 

frame:  11141

*** No instances to display *** 

frame:  11142

*** No instances to display *** 

frame:  11143

*** No instances to display *** 

frame:  11144

*** No instances to display *** 

frame:  11145

*** No instances to display *** 

frame:  11146

*** No instances to display *** 

frame:  11147

*** No instances to display *** 

frame:  11148

*** No instances to